# Question
1. Use RAND to generate synthetic dataset
2. Choose any ANN dataset kaggle or uci machine learning repository

For both, 
1. Implement step-by-step for each with a standard set of weights etc
2. Show tabular representation of hyperparameters as they are tuned at the end. Use CSVLogger code from earlier

Note: <br>
Decide activation functions as well, and define the neural network architecture <br>
Define weights etc, create sub functions for forward and backward propagations <br>
Set a number of iterations and run through the architecture accordingly <br>

#### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from ucimlrepo import fetch_ucirepo 

#### ANN Definition

In [ ]:
# def sigmoid(x):
#     return 1/(1 + np.exp(-x))


# def der_sigmoid(x):
#     return x * (1 - x)


# class my_ANN():
#     def __init__(self, learning_rate=0.001, n_hidden_layers=3, weights=None, random_state=42, output_dim=1, neurons_per_layer=[10, 10, 1]):
#         self.learning_rate = learning_rate
#         self.n_hidden_layers = n_hidden_layers
#         self.weights = weights
#         self.neurons_per_layer = neurons_per_layer
#         self.random_state = random_state
#         self.biases = np.random.uniform(size=(output_dim))
#         self.loss_history = []
    

#     def set_weights(self, x):
#         if self.weights is None:
#             inp_dim = x.shape[1]
#             self.weights = [np.random.uniform(low=-0.01, high=0.01, size=(inp_dim, self.neurons_per_layer[0]))]
#             for i in range(1, self.n_hidden_layers):
#                 self.weights.append(np.random.uniform(low=-0.01, high=0.01, size=(self.neurons_per_layer[i - 1], self.neurons_per_layer[i])))  # Initialize weights for subsequent hidden layers
#         return self.weights
    

#     def forward_propagation(self, x):
#         hidden_layers_output = [x]
        
#         for i in range(self.n_hidden_layers):
#             dot_prod = np.dot(hidden_layers_output[-1], self.weights[i]) + self.biases
#             output = sigmoid(dot_prod)
#             hidden_layers_output.append(output)

#         # final_output = hidden_layers_output[-1]
#         final_output = sigmoid(dot_prod)
#         return hidden_layers_output, final_output


#     def backward_propagation(self, x, y, hidden_layers_output):
#         del_weights = [None] * self.n_hidden_layers
        
#         # Calculate error
#         error = y - hidden_layers_output[-1]

#         # Calculate deltas for each layer
#         del_output_error = error
#         for i in reversed(range(self.n_hidden_layers)):
#             del_dot_prod_error = der_sigmoid(hidden_layers_output[i + 1])
#             del_weights[i] = np.dot(hidden_layers_output[i].T, del_output_error * del_dot_prod_error)
#             print(del_output_error, self.weights[i].T.shape, del_dot_prod_error)
#             del_output_error = np.dot(del_output_error, self.weights[i].T) * del_dot_prod_error

#         return del_weights, del_output_error
    

#     def cross_entropy_loss(self, y, final_output):
#         # Calculate cross-entropy loss
#         epsilon = 1e-15  # Small value to prevent division by zero
#         final_output = np.clip(final_output, epsilon, 1 - epsilon)
#         loss = -np.mean(y * np.log(final_output) + (1 - y) * np.log(1 - final_output))
#         return loss


#     def train(self, x, y, epochs=30):
#         np.random.seed(self.random_state)
#         self.weights = self.set_weights(x)
#         for _ in range(epochs):
#             # Forward propagation
#             hidden_layers_output, final_output = self.forward_propagation(x)
            
#             # Calculate error
#             loss = self.cross_entropy_loss(y, final_output)
#             self.loss_history.append(loss)

#             # Backward propagation
#             del_weights, del_output_error = self.backward_propagation(x, y, hidden_layers_output)
            
#             # Updating weights and bias
#             for i in range(self.n_hidden_layers):
#                 self.weights[i] += self.learning_rate * del_weights[i]
#                 self.biases[i] += self.learning_rate * np.sum(del_output_error * der_sigmoid(hidden_layers_output[i + 1], axis=0))

#             self.weights[-1] += self.learning_rate * np.dot(hidden_layers_output[-2].T, del_output_error * der_sigmoid(final_output))
#             self.biases[-1] += self.learning_rate * np.sum(del_output_error * der_sigmoid(final_output), axis=0)
          
            
#     def plot_loss(self):
#         plt.plot(self.loss_history)
#         plt.xlabel('Epoch')
#         plt.ylabel('Loss')
#         plt.title('Training Loss Over epochs')
#         plt.show()

The above was a failure, I'm trying with a new one below

In [9]:
class ANN2():
    # Creating an object will initialize the network
    def __init__(self, n_input_cols, n_hidden, n_output_cols):
        self.network = list()
        hidden_layer = [{'weights':[np.random.uniform(low=-0.01, high=0.01) for _ in range(n_input_cols + 1)]} for _ in range(n_hidden)]
        self.network.append(hidden_layer)
        output_layer = [{'weights':[np.random.uniform(low=-0.01, high=0.01) for _ in range(n_hidden + 1)]} for _ in range(n_output_cols)]
        self.network.append(output_layer)


    def forward_propagate(self, row):
        def activate(weights, inputs):
            activation = weights[-1]
            for i in range(len(weights) - 1):
                activation += weights[i] * inputs[i]
            return activation

        def neuron_transfer(activation):
            return 1/(1 + np.exp(-activation))
        
        inputs = row
        for layer in self.network:
            new_input = []
            for neuron in layer:
                activation = activate(neuron['weights'], inputs)
                neuron['output'] = neuron_transfer(activation)
                new_input.append(neuron['output'])
            inputs = new_input
        return inputs


#### Synthetic Dataset ANN

In [3]:
n_samples = 13000
n_features = 10
n_classes = 2

x_syn, y_syn = make_classification(n_samples=n_samples, n_features=n_features, n_classes=n_classes, n_informative=6, n_redundant=4, random_state=42)

x_train, x_test, y_train, y_test = train_test_split(x_syn, y_syn, test_size=0.2, random_state=42)

In [12]:
np.random.seed(0)
ann = ANN2(x_train.shape[1], int(2/3 * x_train.shape[1]) + 2, 2) # Should be number of columns in input x, number of neurons per hidden layer (arbitrary, and we only have one hidden layer rn), and number of possible outputs (binary classification = 2)

print("Hidden Layer:")
for neuron in ann.network[0]:
    print(len(neuron['weights']), neuron)

print("\nOutput Layer:")
for neuron in ann.network[1]:
    print(len(neuron['weights']), neuron)

# Testing with a row
row = x_train[0]
output = ann.forward_propagate(row)
print(f"\nOutput for row 0: {output}")



Hidden Layer:
11 {'weights': [0.0009762700785464953, 0.0043037873274483895, 0.0020552675214328773, 0.0008976636599379376, -0.0015269040132219053, 0.002917882261333122, -0.0012482557747461494, 0.007835460015641596, 0.009273255210020587, -0.0023311696234844456, 0.00583450076165329]}
11 {'weights': [0.000577898395058089, 0.0013608912218786469, 0.00851193276585322, -0.00857927883604226, -0.008257414005969186, -0.009595632051193485, 0.0066523969109587595, 0.005563135018997009, 0.007400242964936384, 0.00957236684465528, 0.005983171284334473]}
11 {'weights': [-0.0007704127549413627, 0.00561058352572911, -0.007634511482621335, 0.0027984204265504766, -0.007132934251819072, 0.008893378340991678, 0.0004369664350014329, -0.0017067612001895275, -0.004708887757907461, 0.005484673788684334, -0.0008769933556690285]}
11 {'weights': [0.0013686789773729707, -0.009624203991272897, 0.0023527099415175407, 0.002241914454448428, 0.0023386799374951386, 0.008874961570292482, 0.0036364059820696674, -0.0028098419

#### Dataset import

In [ ]:
# fetch dataset 
dry_bean_dataset = fetch_ucirepo(id=602) 

# data (as pandas dataframes) 
x_uci = dry_bean_dataset.data.features 
y_uci = dry_bean_dataset.data.targets 


In [ ]:
print(x_uci.shape)

In [ ]:
print(x_uci[:1]) 
print(y_uci[:1]) 